In [11]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd
import numpy as np
import yfinance as yf

data = yf.download("NVDA", start="2020-06-01", end="2025-12-28")
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Ensure columns match exactly
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

pl_pct = []
win_stats = []
loss_stats = []

def bollinger_bands(close_array, period=20, std_dev=2):
    close = pd.Series(close_array)
    middle = close.rolling(period).mean()
    std = close.rolling(period).std()
    upper = middle + std_dev * std
    lower = middle - std_dev * std
    diff = upper - lower
    return middle, upper, lower, diff

class Ma50Slope(Strategy):
    ma_period = 18
    smooth_period = 5
    margin_pct = 0.035
    hold_days = 10
    slope = 0.00
    max5_slope = 1
    # bb_period = 20   # Optional: make customizable
    # bb_std = 2 
    max_ma_spread = 1
    min_ma_spread = 0.00
    third_pct_min = 1.02
    half_pct_min = 1.01
    three_qtr_pct_min = 0.995

    def init(self):

        self.sma1 = self.I(SMA, self.data.Close, 10)
        self.sma2 = self.I(SMA, self.data.Close, 20)
        self.sma3 = self.I(SMA, self.data.Close, 5)


        def ema(array, period):
            import pandas as pd
            return pd.Series(array).ewm(span=period, adjust=False).mean().values
        
        self.ma_base = self.I(SMA, self.data.Close, self.ma_period)
        self.ma_smooth = self.I(ema, self.ma_base, self.smooth_period)
        
        # self.b_bands = self.I(bollinger_bands, self.data.Close, self.bb_period, self.bb_std)

    
    def if_early_tp(self):
        current_trade = self.trades[-1]
        bars_since_entry = len(self.data) - current_trade.entry_bar
        past_third = bool(bars_since_entry >= self.hold_days / 3)
        past_half = bool(bars_since_entry >= self.hold_days / 2)
        past_three_qtr = bool(bars_since_entry >= self.hold_days * 0.75)

        if current_trade.is_long and not past_three_qtr:
            pct_diff = self.data.High[-1] / current_trade.entry_price
        elif current_trade.is_long and past_three_qtr:
            pct_diff = self.data.Low[-1] / current_trade.entry_price
        elif current_trade.is_short and not past_three_qtr:
            pct_diff = 2 - (self.data.Low[-1] / current_trade.entry_price)
        else:
            pct_diff = 2 - (self.data.High[-1] / current_trade.entry_price)

        ready_for_tp = bool((past_third and pct_diff >= self.third_pct_min) or
                            (past_half and pct_diff >= self.half_pct_min) or
                            (past_three_qtr and pct_diff >= self.three_qtr_pct_min))
        if ready_for_tp:
            return True
        
    def next(self):
        if len(self.data) < self.ma_period + self.smooth_period + 1:
            return

        ma_current = self.ma_base[-1]
        ma_prev = self.ma_base[-2]
        ma_spread = abs(self.sma3[-1] / self.sma2[-1] - 1)
        sma5_slope = self.sma3[-1] / self.sma3[-2]
        no_vol = np.sum(self.sma3[-5:-1]) / np.sum(self.sma2[-5:-1])
        # if len(self.data) > 485:
        #     print(len(self.data), np.sum(self.sma3[-5:-1]) / np.sum(self.sma2[-5:-1]))

        # if not self.position and ma_spread < self.max_ma_spread and ma_spread > self.min_ma_spread:
        #     if ma_current / ma_prev > (1 + self.slope) and sma5_slope > 1 and no_vol > 1.05:
        #         self.buy()
        #     elif ma_current / ma_prev < (1 - self.slope) and sma5_slope < 1 and no_vol < 0.95:
        #         self.sell()

        # if len(self.data) > 515:
        #     print(len(self.data), sma5_slope)

        if not self.position:
            if ma_current / ma_prev > (1 + self.slope) and sma5_slope > 1:
                self.buy()
            elif ma_current / ma_prev < (1 - self.slope) and sma5_slope < 1:
                self.sell()

        elif self.position and (len(self.trades) > 0):
            current_trade = self.trades[-1]
            bars_since_entry = len(self.data) - current_trade.entry_bar

            past_third = bool(bars_since_entry >= self.hold_days / 3)
            past_half = bool(bars_since_entry >= self.hold_days / 2)
            past_three_qtr = bool(bars_since_entry >= self.hold_days * 0.75)
            


            if past_three_qtr:
                limit_pct = self.three_qtr_pct_min
            elif past_half:
                limit_pct = self.half_pct_min
            elif past_third:
                limit_pct = self.third_pct_min

            if bars_since_entry >= self.hold_days:
                pl_pct.append(current_trade.pl_pct)
                current_trade.close()
            # elif current_trade.is_long and self.if_early_tp(): 
            #     pl_pct.append(current_trade.pl_pct)
            #     current_trade.close()
            # elif current_trade.is_short and self.if_early_tp():
            #     pl_pct.append(current_trade.pl_pct)
            #     current_trade.close()
        

bt = Backtest(data, Ma50Slope, commission=.00,
              exclusive_orders=True,
              finalize_trades=True)
stats = bt.run()
pl_array = np.array(pl_pct)

print(np.sum(pl_array > -0.035) / float(stats["# Trades"])*100, "%")
print(stats)

bt.plot()

/tmp/ipykernel_4101/1096833337.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("NVDA", start="2020-06-01", end="2025-12-28")
[*********************100%***********************]  1 of 1 completed
                                                       

71.55172413793103 %
Start                     2020-06-01 00:00:00
End                       2025-12-26 00:00:00
Duration                   2034 days 00:00:00
Exposure Time [%]                    90.65621
Equity Final [$]                  11938.12545
Equity Peak [$]                   20575.04626
Return [%]                           19.38125
Buy & Hold Return [%]              1988.25163
Return (Ann.) [%]                     3.23542
Volatility (Ann.) [%]                51.74178
CAGR [%]                              2.21907
Sharpe Ratio                          0.06253
Sortino Ratio                         0.10102
Calmar Ratio                          0.04649
Alpha [%]                          -178.63657
Beta                                  0.09959
Max. Drawdown [%]                   -69.58729
Avg. Drawdown [%]                   -15.21428
Max. Drawdown Duration     1325 days 00:00:00
Avg. Drawdown Duration      164 days 00:00:00
# Trades                                  116
Win Rate [%]  

GridPlot(id='p5510', ...)